In [18]:
import numpy as np
import pandas as pd 
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

In [19]:
df = pd.read_csv("./playground-series-s4e9/train.csv",index_col='id')
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [20]:
df.isna().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [21]:
df["fuel_type"].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

# Custom transformations automisation

The following class will preform automizations to the data.
It will be compatible with the scikit learn Pipeline and Transformer objects, making sure the entire pipeline is done automatically on the test data as well.

## Handling Null values in the fuel_type and accident attributes

Since the number of NUlls is very low (3%, 1.5% of the data respectively) we have 2 options:

Option 1 - drop those entries and train the model without them.

Option 2 - Replacing them with another value:
1. The most frequent value in the dataset.
2. A value indicating "unknown".

## Handling Null values in the clean_title attribute

Since the number of NUlls is more significant, we have 2 different options:

Option 1 - drop those entries, since the column would only have one value remaining, it would not contribute to the model training - we would have to drop the attribute entirely.

Option 2 - Replacing them with another value - "No".

We will choose option 2.

In [22]:
class EngineTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # No fitting necessary for this transformer
        return self

    def transform(self, X):
        X = X.copy()
        if 'engine' in X.columns:      
            # Extract values using regular expressions:
                # Extracts the float before 'HP'
            X['horse_power'] = X['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
                # Extracts the float before 'L'
            X['tank_size'] = X['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
                # Extracts the float (to handle NaN) before 'Cylinder'  
            X['num_cylinders'] = X['engine'].str.extract(r'(\d+)\sCylinder').astype(float)    
            # Drop the original 'engine' column
            X.drop('engine', axis=1, inplace=True)  

        return X

In [23]:
class TransmissionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # No fitting necessary for this transformer
        return self

    def transform(self, X):
        X = X.copy()
        # Initialize all attributes to False
        X['automatic'] = False
        X['manual'] = False
        X['dual'] = False
        
		# Categorize as 'dual' if it contains 'Dual'
        df.loc[df['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True
        # Categorize as 'automatic' if it contains AT, A/T, CVT, or Automatic (but not Dual)
        df.loc[(df['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) &~df['dual'],'automatic'] = True
		# Categorize as 'manual' if it contains MT, M/T, or Manual (but not Dual)
        df.loc[(df['transmission'].str.contains(r'\b(MT|M/T|Manual)\b', case=False, na=False)) &~df['dual'],'manual'] = True

        return X

In [24]:
class ImputerOne(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # No fitting necessary for this transformer
        return self

    def transform(self, X):
        X = X.copy()
        if 'fuel_type' in X.columns:
            X = X[(X['fuel_type'] != 'not supported')]
            X = X[(X['fuel_type'] != '–')]
            X = X.dropna(subset=['fuel_type'])
        if 'accident' in X.columns:
            X = X.dropna(subset=['accident'])
        if 'clean_title' in X.columns:
            X = X['clean_title'].fillna("No")
        
        return X

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class ImputerTwo(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # No fitting necessary for this transformer
        return self

    def transform(self, X):
        X = X.copy()
        if 'fuel_type' in X.columns:
            X["fuel_type"] = X["fuel_type"].replace('not supported', "Gasoline")
            X["fuel_type"] = X["fuel_type"].replace('–', "Gasoline")
            X["fuel_type"] = X["fuel_type"].replace(pd.NA, "Gasoline")
        if 'accident' in X.columns:
            X = X['accident'].fillna("Unknown")
        if 'clean_title' in X.columns:
            X = X['clean_title'].fillna("No")
        return X
            

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression

In [27]:
preprocessor_1 = ColumnTransformer(
    transformers=[
        ('engine', EngineTransformer(), ['engine']),
		('transmission', TransmissionTransformer(), ['transmission']),
        ('imputer_1', ImputerOne(), ['fuel_type', 'accident', 'clean_title'])
    ])

prepro1_df = preprocessor_1.fit_transform(df)

/tmp/ipykernel_7358/1806102230.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True
/tmp/ipykernel_7358/1806102230.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(df['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) &~df['dual'],'automatic'] = True


TypeError: bad operand type for unary ~: 'float'

In [ ]:
# Create a second preprocessor like the first, but with the second imputer

prepro2_df = preprocessor_2.fit_transform(df)

In [ ]:
# Create the pipeline
model = Pipeline(steps=[
    #('preprocessor', preprocessor_1),
    ('regressor', LinearRegression())
])

# Fit the pipeline to the data
X = df.drop(columns=['price'])
y = df['price']
model.fit(X, y)

In [ ]:
# Create the pipeline
pipeline_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_2),
    ('regressor', LinearRegression())
])

# Fit the pipeline to the data
X = df.drop(columns=['price'])
y = df['price']
pipeline_2.fit(X, y)

In [3]:
# preprecoess the color columns